# Hard Voting Ensemble

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
np.set_printoptions(threshold=np.inf)

###### ensemble csv files list ######
files_path = [
    '/opt/ml/notebooks/7006.csv',
    '/opt/ml/notebooks/7015.csv',
    '/opt/ml/notebooks/7024.csv',
    '/opt/ml/notebooks/7055.csv',
    '/opt/ml/notebooks/7068.csv'
    ]
###### ensemble csv files list ######

###### 저장 위치 #######
save_file = "./ensemble_convknet.csv"

In [7]:
pandas_list = []
predict_list = []
for file_path in files_path:
    pandas_list.append(pd.read_csv(file_path))

pandas_stack = np.empty(shape=(1,819,256*256), dtype = np.int8)
for pandas in tqdm(pandas_list, desc ='outer'):
    tmp = np.empty(shape=(1,256*256), dtype = np.int8)
    for i in tqdm(range(len(pandas)), desc = "inner"):
        tmp = np.append(tmp, np.array(list(map(int, pandas['PredictionString'][i].split()))).reshape(1,256*256), axis = 0)
    tmp = np.delete(tmp,0, axis = 0)
    pandas_stack = np.append(pandas_stack, tmp.reshape(1,819,256*256), axis = 0)
pandas_stack = np.delete(pandas_stack,0, axis = 0)
print(f" stack numpy shape (앙상블 파일 수 , 819, 65536) :{pandas_stack.shape}")

b = np.apply_along_axis(lambda x: np.bincount(x, minlength=11), axis=0, arr=pandas_stack)
b = np.apply_along_axis(np.argmax, 0, b)
print(f"최빈값 구한 shape (819, 65536): {b.shape}")
print()

print("문자열 길이 = 데이터(256*256) + 공백((256*256 -1) = 131071")
print(f"원본 pandas 문자열 길이: {len(pandas_list[0]['PredictionString'][0])}")
print("생성 pandas 문자열 길이:", len(np.array2string(b[0]).replace('\n','')[1:-1]))
for i in tqdm(range(len(pandas_list[0]))):
    pandas_list[0]['PredictionString'][i] = np.array2string(b[i]).replace('\n','')[1:-1]
    
pandas_list[0].to_csv(save_file)

outer: 100%|██████████| 5/5 [09:03<00:00, 108.62s/it]


 stack numpy shape (앙상블 파일 수 , 819, 65536) :(5, 819, 65536)


  0%|          | 0/819 [00:00<?, ?it/s]

최빈값 구한 shape (819, 65536): (819, 65536)

문자열 길이 = 데이터(256*256) + 공백((256*256 -1) = 131071
원본 pandas 문자열 길이: 131071
생성 pandas 문자열 길이: 131071


<ipython-input-7-f764df6c339b>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pandas_list[0]['PredictionString'][i] = np.array2string(b[i]).replace('\n','')[1:-1]
100%|██████████| 819/819 [01:24<00:00,  9.66it/s]


In [11]:
len(pandas_list[0]['PredictionString'][0])

131071